<a href="https://colab.research.google.com/github/maevezhw/web-scraping/blob/main/Data%20Tidak%20Terstruktur%20-%20Twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Library

In [ ]:
!pip install snscrape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 3.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import snscrape.modules.twitter as sntwitter

Scraping Tweet

In [ ]:
scraper = sntwitter.TwitterSearchScraper("rekomendasi jogja")
tweets = []

In [ ]:
# mengambil data satu per satu menggunakan bantuan library
for i, tweet in enumerate(scraper.get_items()):
  data = [
    tweet.date,
    tweet.user.username,
    tweet.content,
    tweet.likeCount,
    tweet.retweetCount,
    tweet.replyCount
]

# memasukkan data ke list tweets
  tweets.append(data)
  if i > 200:
    break

# menampilkan data
tweets

ERROR:snscrape.base:Error retrieving https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&include_ext_has_nft_avatar=1&include_ext_is_blue_verified=1&include_ext_verified_type=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_ext_limited_action_results=false&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_ext_collab_control=true&include_ext_views=true&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&include_ext_sensitive_media_warning=true&include_ext_trusted_friends_metadata=true&send_error_codes=true&simple_quoted_tweet=true&q=rekomendasi+jogja&tweet_search_mode=live&count=20&query_source=spelling_expansion_revert_click&pc=1&spelling_corrections=1&include_ext_edit_control=true&ext=m

ScraperException: ignored

In [ ]:
# merubah list menjadi dataframe
tweets_df = pd.DataFrame(tweets, columns=["date", "username", "content", "like", "retweet", "reply"])
tweets_df

Preprocessing

In [ ]:
# memecah kolom date yang tadinya berisi tanggal dan waktu menjadi terpisah date dan time
import datetime
tweets_df["time"] = pd.to_datetime(tweets_df["date"]).dt.time
tweets_df["date"] = pd.to_datetime(tweets_df["date"]).dt.date

# merapikan urutan kolom pada dataframe
tweets_df = tweets_df[["date", "time", "username", "content", "like", "retweet", "reply"]]

# data pada konten dibuat semuanya menjadi lowercase
tweets_df["content"] = tweets_df["content"].str.lower()

# membuat satu kolom baru untuk mengetahui jenis rekomendasi dengan cara :
# mempartisi data konten menjadi dua, yakni bagian sebelum tulisan 'rekomendasi' dan setelah tulisan 'rekomendasi'
tweets_df["jenis rekomendasi"] = tweets_df["content"].str.partition("rekomendasi")[2]

# menghilangkan unsur angka pada data teks
tweets_df["jenis rekomendasi"] = tweets_df["jenis rekomendasi"].str.replace('\d+', '')

# menghapuskan spasi sebelum teks dan memisahkan teks menjadi per kata
tweets_df["jenis rekomendasi"] = tweets_df["jenis rekomendasi"].str.lstrip()
tweets_df_rekomendasi = tweets_df["jenis rekomendasi"].str.split(" ")

# mengambil hanya satu kata di belakang kata 'rekomendasi' untuk mengetahui jenis rekomendasi
tweets_df_r = []

for i in range(0, tweets_df_rekomendasi.size):
  tweets_df_r.append(tweets_df_rekomendasi[i][0])

tweets_df["jenis rekomendasi"] = tweets_df_r


In [ ]:
tweets_df.describe()

In [ ]:
from google.colab import files
tweets_df.to_csv('tweets_scraping.csv')
files.download('tweets_scraping.csv')

In [ ]:
tweets_df.loc[tweets_df["username"]=="BioskoponlineID"]